## Deployment of Skin Lesion Model

In [1]:
#import libraries
import keras
import tensorflow as tf
from keras.preprocessing import image
import json
import requests
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.metrics import top_k_categorical_accuracy
from keras.models import load_model
from keras.optimizers import Adam

In [2]:
#need to downgrade to 2.2.0 at the moment so that the model accuracy is the same
#saving model doesn't seem to work on tf ver. 2.3.1

#!pip install 'tensorflow == 2.2.0'
print(tf.__version__)

2.2.0


## Saving Model

In [6]:
#run this code for top_2_accuracy metric for modelling

def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

In [8]:
#load model 4a
model_4a = load_model('../model/lesion_classifier/4a_cnn_smote_model_1.h5', 
                    custom_objects={'top_2_accuracy': top_2_accuracy}) 

In [9]:
#loading our model's best weights
model_4a.load_weights("../weights/lesion/weights-improvement-14-0.71.hdf5")

In [10]:
#save version 1 of model 4a
model_4a.save('../deployment/lesion/1')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./deployment/lesion/1/assets


Deployment of the model is done through Tensorflow Serving on Docker. Installation of Docker is required.

### Importing image and its label to put into model

In [11]:
images = []
img = image.load_img('../datasets/processed_image/ISIC_0024356.jpg', target_size=(284, 284))
img_array = image.img_to_array(img)
img = img_array/255
images.append(img)
x = np.array(images)

In [14]:
#load in labeled_ground_truth
testing_labels = pd.read_csv('../datasets/deployment/testing_labels.csv')

In [15]:
#create list of columns to be excluded from target variables y
columns_dropped=['image','lesion', 'diagnosis', 'benign', 'malignant', 'precancerous']

#create y that contains all 3 target variables/classes
y = np.array(testing_labels.drop(columns= columns_dropped))

In [16]:
y_true = np.argmax(y, axis=1)
y_true

array([1])

### Running the server

In [25]:
data = json.dumps({"instances": x.tolist()})
class_names=['mel', 'nv', 'bcc', 'akiec', 'bkl', 'df', 'vasc']
headers = {"content-type": "application/json"}

json_response = requests.post('http://localhost:8501/v1/models/lesion:predict',
                              data=data,
                              headers=headers)

predictions = json.loads(json_response.text)


print('The model predicted this was a {} (class {})'.format(class_names[np.argmax(predictions)], np.argmax(predictions)))
print('It was actually {} (class {})'.format(class_names[int(y_true)], int(y_true)))

The model predicted this was a mel (class 0)
It was actually nv (class 1)
